#Corporación Favorita Grocery Sales Forecasting
The objective is to predict future sales of items in Favorita grocery stores across different regions of Ecuador. So that Reliable predictions can be helpful to optimize inventory management, prevent stockouts, and improve promotion strategies.

#1- Importing Libraries

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [24]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# 2- Data Retreival

In [ ]:

df_stores = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/stores.csv')
df_items = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/items.csv')
df_holidays_event = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/holidays_events.csv')
df_sample_submission = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/sample_submission.csv')
df_oil = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/oil.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/test.csv')
df_transactions = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/transactions.csv')
df_train_Guayas = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/train_Guayas.csv')


<ipython-input-3-1043019830>:8: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_Guayas = pd.read_csv('/content/drive/MyDrive/Projects/Timeseries_project/data/train_Guayas.csv')


In [ ]:
# Read data locally 

file_path = '/Users/D065623/Documents/Amna/Time_series_project/Data/'

df_stores = pd.read_csv(file_path + 'stores.csv')
df_items = pd.read_csv(file_path + 'items.csv')
df_holidays_event = pd.read_csv(file_path + 'holidays_events.csv')
df_oil = pd.read_csv(file_path + 'oil.csv')
df_train_Guayas = pd.read_csv(file_path + 'train_Guayas.csv')

/var/folders/c8/sh5pt5vd6l18mnnz089s6hd80000gn/T/ipykernel_39694/1067542697.py:9: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train_Guayas = pd.read_csv(file_path + 'train_Guayas.csv')


In [ ]:
df_train = df_train_Guayas.copy()
df_train.shape

(14745768, 6)

# 3- Data Preprocessing

In [ ]:
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,18790,2013-01-02,24,105574,12.0,NaN
1,18791,2013-01-02,24,105575,2.0,NaN
2,18792,2013-01-02,24,105577,1.0,NaN
3,18793,2013-01-02,24,105693,7.0,NaN
4,18794,2013-01-02,24,105737,17.0,NaN


## 3.1- Checking Missing values

In [ ]:
df_train.isnull().sum() #show the count of missing values

id                   0
date                 0
store_nbr            0
item_nbr             0
unit_sales           0
onpromotion    2747275
dtype: int64

In [ ]:
df_train['onpromotion'].unique()

array([nan, False, True], dtype=object)

In [ ]:

df_train['onpromotion'].value_counts()

onpromotion
False    11319196
True       679297
Name: count, dtype: int64

In [ ]:
df_train['onpromotion'].fillna(False, inplace=True) #nulls value replaced and column type changed
df_train['onpromotion']= df_train['onpromotion'].astype(bool)
df_train.isnull().sum()

/var/folders/c8/sh5pt5vd6l18mnnz089s6hd80000gn/T/ipykernel_39694/194577022.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['onpromotion'].fillna(False, inplace=True) #nulls value replaced and column type changed
/var/folders/c8/sh5pt5vd6l18mnnz089s6hd80000gn/T/ipykernel_39694/194577022.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
dtype: int64

In [ ]:
df_train['onpromotion'].unique()

array([False,  True])

In [ ]:
df_train['onpromotion'].value_counts()

onpromotion
False    14066471
True       679297
Name: count, dtype: int64

## 3.2- Replacing negative unit sales

In [ ]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,14745768.0,6.151701e+07,3.639139e+07,18790.0,29851028.75,60877412.0,92995259.25,125491523.0
store_nbr,14745768.0,3.245622e+01,7.850337e+00,24.0,27.00,30.0,35.00,51.0
item_nbr,14745768.0,9.031913e+05,5.233564e+05,96995.0,422452.00,886398.0,1209721.00,2127114.0
unit_sales,14745768.0,6.814486e+00,1.889631e+01,-10002.0,2.00,3.0,7.00,17146.0


In [ ]:
df_train.loc[df_train['unit_sales']<0,'unit_sales'] = 0

## 3.3- Outlier detection

In [ ]:
# calculate z_score
def calculate_Z_score(group):
    mean_sales = group['unit_sales'].mean()
    std_sales = group['unit_sales'].std()
    z_scores = (group['unit_sales'] - mean_sales) / std_sales if std_sales != 0 else 1
    group['z_score'] = z_scores
    return group

In [ ]:
df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr']).apply(calculate_Z_score)
df_train_grouped.head()

/var/folders/c8/sh5pt5vd6l18mnnz089s6hd80000gn/T/ipykernel_39694/2064882231.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr']).apply(calculate_Z_score)


id        date  store_nbr  item_nbr  unit_sales  \
store_nbr item_nbr                                                              
24        96995    34742  302230  2013-01-09         24     96995         2.0   
                   48648  419989  2013-01-12         24     96995         2.0   
                   68758  582436  2013-01-16         24     96995         1.0   
                   73699  622071  2013-01-17         24     96995         2.0   
                   88770  745650  2013-01-20         24     96995         3.0   

                          onpromotion   z_score  
store_nbr item_nbr                               
24        96995    34742        False  0.053571  
                   48648        False  0.053571  
                   68758        False -0.604164  
                   73699        False  0.053571  
                   88770        False  0.711307

In [ ]:
#finding outlier using thresh_hold z_score > 5
outlier = df_train_grouped[df_train_grouped['z_score']>5]
outlier






id        date  store_nbr  item_nbr  \
store_nbr item_nbr                                                        
24        96995    2029557    15666702  2013-12-18         24     96995   
          99197    7241112    59775862  2015-10-20         24     99197   
                   7303849    60322807  2015-10-26         24     99197   
          103520   2036242    15717753  2013-12-19         24    103520   
                   2415492    18886543  2014-02-13         24    103520   
...                                ...         ...        ...       ...   
51        2048609  13492066  114286905  2017-05-01         51   2048609   
          2054291  12349340  104195298  2017-01-25         51   2054291   
          2057033  14396173  122354227  2017-07-16         51   2057033   
          2058907  13800316  117045510  2017-05-27         51   2058907   
          2068536  13006731  109972933  2017-03-21         51   2068536   

                             unit_sales  onpromotion    z_score  
store_nbr item_nbr                                               
24        96995    2029557         14.0        False   7.946399  
          99197    7241112         14.0        False   5.822016  
                   7303849         19.0        False   8.276173  
          103520   2036242         27.0        False   6.642670  
                   2415492         22.0        False   5.154400  
...                                 ...          ...        ...  
51        2048609  13492066        48.0        False   7.830664  
          2054291  12349340        21.0        False   6.141217  
          2057033  14396173        21.0         True   5.247882  
          2058907  13800316        12.0        False   5.054512  
          2068536  13006731       374.0         True  11.238102  

[48485 rows x 7 columns]

considering the count of these entries, it is indicated that probably they are just big sales and not be cosidered as outliers.

In [ ]:
del(df_train_grouped)

## 3.4- Dealing with missing dates with zero sales

In [ ]:
#get the all posible dates in the range of df_traine['date'] min and max values
df_train['date'] = pd.to_datetime(df_train['date'])
min_date = df_train['date'].min()
max_date = df_train['date'].max()
df_full_date_range = pd.date_range(start=min_date, end=max_date, freq='D')
df_full_date_range = pd.DataFrame({'date': df_full_date_range})
df_full_date_range.head()

,date
0,2013-01-02
1,2013-01-03
2,2013-01-04
3,2013-01-05
4,2013-01-06


In [ ]:
#combing two data frame brings us all possible options of a date for the given store and item
store_item_combinations = df_train[['store_nbr', 'item_nbr']].drop_duplicates()
df_cross_join = pd.merge(store_item_combinations, df_full_date_range, how='cross')
df_cross_join.head()

,store_nbr,item_nbr,date
0,24,105574,2013-01-02
1,24,105574,2013-01-03
2,24,105574,2013-01-04
3,24,105574,2013-01-05
4,24,105574,2013-01-06


In [ ]:
del(df_full_date_range)
del(store_item_combinations)

In [ ]:
date_fixed_df = pd.merge(df_cross_join, df_train, on=['store_nbr', 'item_nbr', 'date'], how='left')
date_fixed_df['unit_sales'].fillna(0, inplace=True)
df_train = date_fixed_df # after fixing tha dates problme, the original training data frame is over written
df_train.head()

/var/folders/c8/sh5pt5vd6l18mnnz089s6hd80000gn/T/ipykernel_39694/2670210681.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  date_fixed_df['unit_sales'].fillna(0, inplace=True)


,store_nbr,item_nbr,date,id,unit_sales,onpromotion
0,24,105574,2013-01-02,18790.0,12.0,False
1,24,105574,2013-01-03,59692.0,1.0,False
2,24,105574,2013-01-04,99664.0,3.0,False
3,24,105574,2013-01-05,140805.0,4.0,False
4,24,105574,2013-01-06,182800.0,7.0,False


## 3.5- Feature Engineering

In [ ]:
# creating time based features
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day
df_train['day_of_week'] = df_train['date'].dt.dayofweek
df_train

,store_nbr,item_nbr,date,id,unit_sales,onpromotion,year,month,day,day_of_week
0,24,105574,2013-01-02,18790.0,12.0,False,2013,1,2,2
1,24,105574,2013-01-03,59692.0,1.0,False,2013,1,3,3
2,24,105574,2013-01-04,99664.0,3.0,False,2013,1,4,4
3,24,105574,2013-01-05,140805.0,4.0,False,2013,1,5,5
4,24,105574,2013-01-06,182800.0,7.0,False,2013,1,6,6
...,...,...,...,...,...,...,...,...,...,...
35069351,51,2123791,2017-08-11,NaN,0.0,NaN,2017,8,11,4
35069352,51,2123791,2017-08-12,NaN,0.0,NaN,2017,8,12,5
35069353,51,2123791,2017-08-13,NaN,0.0,NaN,2017,8,13,6
35069354,51,2123791,2017-08-14,NaN,0.0,NaN,2017,8,14,0


In [ ]:
#creating rolling Avg
df_train['unit_sales_7d_avg'] = df_train.groupby(['item_nbr', 'store_nbr'])['unit_sales'].transform(lambda x: x.rolling(window=7).mean())


df_train.head(15)

,store_nbr,item_nbr,date,id,unit_sales,onpromotion,year,month,day,day_of_week,unit_sales_7d_avg
0,24,105574,2013-01-02,18790.0,12.0,False,2013,1,2,2,NaN
1,24,105574,2013-01-03,59692.0,1.0,False,2013,1,3,3,NaN
2,24,105574,2013-01-04,99664.0,3.0,False,2013,1,4,4,NaN
3,24,105574,2013-01-05,140805.0,4.0,False,2013,1,5,5,NaN
4,24,105574,2013-01-06,182800.0,7.0,False,2013,1,6,6,NaN
5,24,105574,2013-01-07,223895.0,3.0,False,2013,1,7,0,NaN
6,24,105574,2013-01-08,263107.0,3.0,False,2013,1,8,1,4.714286
7,24,105574,2013-01-09,302232.0,7.0,False,2013,1,9,2,4.000000
8,24,105574,2013-01-10,341031.0,3.0,False,2013,1,10,3,4.285714
9,24,105574,2013-01-11,379570.0,6.0,False,2013,1,11,4,4.714286


# 4- Creating CSV of preprocessed Train of Guayas

In [25]:
print(f'df_train shape: {df_train.shape}')

df_train shape: (35069356, 11)


In [26]:
#save_fileName = '/content/drive/MyDrive/Projects/Timeseries_project/data/train_Guayas_preprocessed.csv'
save_fileName = file_path + 'train_Guayas_preprocessed.csv'
df_train.to_csv(save_fileName, index=False)
print(f'File saved to {save_fileName}')

File saved to /Users/D065623/Documents/Amna/Time_series_project/Data/train_Guayas_preprocessed.csv
